**Name: Shaurya Gupta**

**Reg. No: 18BLC1092**

**Course: Data analytics and visualisation**

***Digital Assignment***

We will we be using the Linear Regression model and california housing dataset to compare the working in pyspark and sklearn.

In [ ]:
!python --version

Python 3.6.9


# **Downloading Pyspark**
Spark version 3.0.1 is used for the following implementation.

In [ ]:
!pwd
!wget https://mirrors.estointernet.in/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

/content
--2020-10-25 15:59:43--  https://mirrors.estointernet.in/apache/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
Resolving mirrors.estointernet.in (mirrors.estointernet.in)... 43.255.166.254, 2403:8940:3:1::f
Connecting to mirrors.estointernet.in (mirrors.estointernet.in)|43.255.166.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 219929956 (210M) [application/octet-stream]
Saving to: ‘spark-3.0.1-bin-hadoop2.7.tgz’

spark-3.0.1-bin-had 100%[===================>] 209.74M  8.00MB/s    in 27s     

2020-10-25 16:00:12 (7.71 MB/s) - ‘spark-3.0.1-bin-hadoop2.7.tgz’ saved [219929956/219929956]



In [ ]:
!tar xvzf spark-3.0.1-bin-hadoop2.7.tgz

spark-3.0.1-bin-hadoop2.7/
spark-3.0.1-bin-hadoop2.7/RELEASE
spark-3.0.1-bin-hadoop2.7/examples/
spark-3.0.1-bin-hadoop2.7/examples/src/
spark-3.0.1-bin-hadoop2.7/examples/src/main/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/FPGrowthExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/GBTExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/ALSExample.scala
spark-3.0.1-bin-hadoop2.7/examples/src/main/scala/org/apache/spark/examples/ml/KMeansExample.scala
spark-3.0.1-bin-hadoop2.7/example

# **Data set**

This is a calafornia housing datset and was obtained from the StatLib repository. It appeared in a 1997 paper titled Sparse Spatial Autoregressions, written by Pace, R. Kelley and Ronald Barry and published in the Statistics and Probability Letters journal. The researchers built this data set by using the 1990 California census data

link - https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cal_housing.domain to cal_housing.domain
Saving cal_housing.data to cal_housing.data


## Installing findspark
findspark is needed to locate and initialise pyspark so that it can be imported as a Pypi package directly.

In [ ]:
!pip install findspark

In [ ]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

import findspark
findspark.init()

## Creating a new Spark Session

In [ ]:
from pyspark.sql import SparkSession

# Build the SparkSession
spark = SparkSession.builder \
   .master("local") \
   .appName("Linear Regression Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

## Loading the uploaded Data

The data contains one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people). In this sample a block group on average includes 1425.5 individuals living in a geographically compact area.

These spatial data contain 20,640 observations on housing prices with 9 economic variables:


**Longitude** refers to the angular distance of a geographic place north or south of the earth’s equator for each block group \
**Latitude** refers to the angular distance of a geographic place east or west of the earth’s equator for each block group \
**Housing median age** is the median age of the people that belong to a block group. Note that the median is the value that lies at the midpoint of a frequency distribution of observed values \
**Total rooms** is the total number of rooms in the houses per block group \
**Total bedrooms** is the total number of bedrooms in the houses per block group \
**Population** is the number of inhabitants of a block group \
**Households** refers to units of houses and their occupants per block group \
**Median income** is used to register the median income of people that belong to a block group \ 
**Median house value** is the dependent variable and refers to the median house value per block group \

**The Median house value is the dependent variable and will be assigned the role of the target variable in your ML model.**



In [ ]:
# Load the data points
rdd = sc.textFile('cal_housing.data')

# Load the header
header = sc.textFile('cal_housing.domain')
"Header", header.collect(), "rdd", rdd.take(2)

('Header',
 ['longitude: continuous.',
  'latitude: continuous.',
  'housingMedianAge: continuous. ',
  'totalRooms: continuous. ',
  'totalBedrooms: continuous. ',
  'population: continuous. ',
  'households: continuous. ',
  'medianIncome: continuous. ',
  'medianHouseValue: continuous. '],
 'rdd',
 ['-122.230000,37.880000,41.000000,880.000000,129.000000,322.000000,126.000000,8.325200,452600.000000',
  '-122.220000,37.860000,21.000000,7099.000000,1106.000000,2401.000000,1138.000000,8.301400,358500.000000'])

## Splitting the Entries

Splitting the entries to make sure that the rows of your data are separate elements.

In [ ]:
rdd = rdd.map(lambda line: line.split(","))

rdd.take(2)

[['-122.230000',
  '37.880000',
  '41.000000',
  '880.000000',
  '129.000000',
  '322.000000',
  '126.000000',
  '8.325200',
  '452600.000000'],
 ['-122.220000',
  '37.860000',
  '21.000000',
  '7099.000000',
  '1106.000000',
  '2401.000000',
  '1138.000000',
  '8.301400',
  '358500.000000']]

## Creating A schema to convert into a DataFrame

In [ ]:
from pyspark.sql import Row

# Map the RDD to a DF
df = rdd.map(lambda line: Row(longitude=line[0], 
                              latitude=line[1], 
                              housingMedianAge=line[2],
                              totalRooms=line[3],
                              totalBedRooms=line[4],
                              population=line[5], 
                              households=line[6],
                              medianIncome=line[7],
                              medianHouseValue=line[8])).toDF()
df.show(), df.columns

+-----------+---------+----------------+-----------+-------------+-----------+-----------+------------+----------------+
|  longitude| latitude|housingMedianAge| totalRooms|totalBedRooms| population| households|medianIncome|medianHouseValue|
+-----------+---------+----------------+-----------+-------------+-----------+-----------+------------+----------------+
|-122.230000|37.880000|       41.000000| 880.000000|   129.000000| 322.000000| 126.000000|    8.325200|   452600.000000|
|-122.220000|37.860000|       21.000000|7099.000000|  1106.000000|2401.000000|1138.000000|    8.301400|   358500.000000|
|-122.240000|37.850000|       52.000000|1467.000000|   190.000000| 496.000000| 177.000000|    7.257400|   352100.000000|
|-122.250000|37.850000|       52.000000|1274.000000|   235.000000| 558.000000| 219.000000|    5.643100|   341300.000000|
|-122.250000|37.850000|       52.000000|1627.000000|   280.000000| 565.000000| 259.000000|    3.846200|   342200.000000|
|-122.250000|37.850000|       52

(None,
 ['longitude',
  'latitude',
  'housingMedianAge',
  'totalRooms',
  'totalBedRooms',
  'population',
  'households',
  'medianIncome',
  'medianHouseValue'])

## Checking the data Types of the data points

In [ ]:
df.printSchema()

root
 |-- longitude: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- housingMedianAge: string (nullable = true)
 |-- totalRooms: string (nullable = true)
 |-- totalBedRooms: string (nullable = true)
 |-- population: string (nullable = true)
 |-- households: string (nullable = true)
 |-- medianIncome: string (nullable = true)
 |-- medianHouseValue: string (nullable = true)



## Conversion into accurate data Variables
We need numerical values and not strings for the analysis for which the conversions have to take place.

In [ ]:
from pyspark.sql.types import *

# Write a custom function to convert the data type of DataFrame columns
def convertColumn(df, names, newType):
  for name in names: 
     df = df.withColumn(name, df[name].cast(newType))
  return df 

columns = ['households', 'housingMedianAge', 'latitude', 'longitude', 'medianHouseValue', 'medianIncome', 'population', 'totalBedRooms', 'totalRooms']

# Conver the `df` columns to `FloatType()`
df = convertColumn(df, columns, FloatType())

df.printSchema()

root
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- housingMedianAge: float (nullable = true)
 |-- totalRooms: float (nullable = true)
 |-- totalBedRooms: float (nullable = true)
 |-- population: float (nullable = true)
 |-- households: float (nullable = true)
 |-- medianIncome: float (nullable = true)
 |-- medianHouseValue: float (nullable = true)



## Summarizing the Data

In [ ]:
df.describe().show()

+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|          longitude|         latitude|  housingMedianAge|        totalRooms|    totalBedRooms|        population|       households|      medianIncome|  medianHouseValue|
+-------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              20640|            20640|             20640|             20640|            20640|             20640|            20640|             20640|             20640|
|   mean|-119.56970444871473|35.63186143109965|28.639486434108527|2635.7630813953488|537.8980135658915|1425.4767441860465|499.5396802325581|3.8706710030346416|206855.81690891474|
| stddev|  2.003531742932898|2.135952380602968| 12.58555761211163|2181.6152515827944| 421.247905943133|  

# Data Pre-Processing
## Scaling The Dependent variable to independent variable ranges

In [ ]:
from pyspark.sql.functions import col
df = df.withColumn("medianHouseValue", col("medianHouseValue")/100000)
df.take(2)

[Row(longitude=-122.2300033569336, latitude=37.880001068115234, housingMedianAge=41.0, totalRooms=880.0, totalBedRooms=129.0, population=322.0, households=126.0, medianIncome=8.325200080871582, medianHouseValue=4.526),
 Row(longitude=-122.22000122070312, latitude=37.86000061035156, housingMedianAge=21.0, totalRooms=7099.0, totalBedRooms=1106.0, population=2401.0, households=1138.0, medianIncome=8.301400184631348, medianHouseValue=3.585)]

## Feature Engineering
Additional Features can be created like the following:\
**Rooms per household** which refers to the number of rooms in households per block group \
**Population per household**, which basically gives you an indication of how many people live in households per block group \
**Bedrooms per total rooms** which will give you an idea about how many rooms are bedrooms per block group \

In [ ]:
roomsPerHousehold = df.select(col("totalRooms")/col("households"))

populationPerHousehold = df.select(col("population")/col("households"))

bedroomsPerRoom = df.select(col("totalBedRooms")/col("totalRooms"))

df = df.withColumn("roomsPerHousehold", col("totalRooms")/col("households")) \
   .withColumn("populationPerHousehold", col("population")/col("households")) \
   .withColumn("bedroomsPerRoom", col("totalBedRooms")/col("totalRooms"))
   
df.first()

Row(longitude=-122.2300033569336, latitude=37.880001068115234, housingMedianAge=41.0, totalRooms=880.0, totalBedRooms=129.0, population=322.0, households=126.0, medianIncome=8.325200080871582, medianHouseValue=4.526, roomsPerHousehold=6.984126984126984, populationPerHousehold=2.5555555555555554, bedroomsPerRoom=0.14659090909090908)

## Standardising All Independent Variables

Leaving out the Dependent Variable Median-House-Value 

Leaving out variables such as longitude, latitude, housingMedianAge and totalRooms.

In [ ]:
from pyspark.ml.linalg import DenseVector

df = df.select("medianHouseValue", 
              "totalBedRooms", 
              "population", 
              "households", 
              "medianIncome", 
              "roomsPerHousehold", 
              "populationPerHousehold", 
              "bedroomsPerRoom")

input_data = df.rdd.map(lambda x: (x[0], DenseVector(x[1:])))

df = spark.createDataFrame(input_data, ["label", "features"])

In [ ]:
from pyspark.ml.feature import StandardScaler

# Initialize the `standardScaler`
standardScaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# Fit the DataFrame to the scaler
scaler = standardScaler.fit(df)

# Transform the data in `df` with the scaler
scaled_df = scaler.transform(df)

# Inspect the result
scaled_df.take(2)

[Row(label=4.526, features=DenseVector([129.0, 322.0, 126.0, 8.3252, 6.9841, 2.5556, 0.1466]), features_scaled=DenseVector([0.3062, 0.2843, 0.3296, 4.3821, 2.8228, 0.2461, 2.5264])),
 Row(label=3.585, features=DenseVector([1106.0, 2401.0, 1138.0, 8.3014, 6.2381, 2.1098, 0.1558]), features_scaled=DenseVector([2.6255, 2.1202, 2.9765, 4.3696, 2.5213, 0.2031, 2.6851]))]

## Splitting into Training and Testing Sets

In [ ]:
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

## Fitting The Linear Regression Model

In [ ]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(labelCol="label", maxIter=1, regParam=0.3, elasticNetParam=0.8) ## elasticNetParam --> alpha for learning rate, regParam --> lambda for regularization parameter

# Fit the data to the model
linearModel = lr.fit(train_data)

## Making Predictions

In [ ]:
predicted = linearModel.transform(test_data)

predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select("label").rdd.map(lambda x: x[0])

predictionAndLabel = predictions.zip(labels).collect()

predictionAndLabel[:5]

[(1.967365690052053, 0.14999),
 (1.9891240427526629, 0.175),
 (1.974234393625167, 0.332),
 (2.0025511782871126, 0.346),
 (2.0058348572001417, 0.35)]

## Evaluating the Model

In [ ]:
# Get the RMSE
linearModel.summary.meanSquaredError

1.2118026357830542

In [ ]:
# Get the R2
linearModel.summary.r2

0.09004899728545235

# Calculating Time With Pyspark

In [ ]:
import time
from pyspark.ml.regression import LinearRegression


lr = LinearRegression(labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8) ## elasticNetParam --> alpha for learning rate, regParam --> lambda for regularization parameter

start = time.time()
# Fit the data to the model
linearModel = lr.fit(train_data)
end = time.time()

predicted = linearModel.transform(test_data)

predictions = predicted.select("prediction").rdd.map(lambda x: x[0])
labels = predicted.select("label").rdd.map(lambda x: x[0])

predictionAndLabel = predictions.zip(labels).collect()

predictionAndLabel[:5]

# Get the RMSE
m1 = linearModel.summary.meanSquaredError

# Get the R2
r1 = linearModel.summary.r2

print("mean_squared_error =", m1, "R_squared =", r1)

sp = end - start

# total time taken
print(f"Runtime of the program is {end - start} seconds")


mean_squared_error = 0.7734029416457108 R_squared = 0.4192463677898063
Runtime of the program is 4.694941997528076 seconds


# Calculating Time Taken Using Sk-Learn
## Pre-processing

In [ ]:
from sklearn import *
import numpy as np
import pandas as pd

header = pd.read_csv('cal_housing.domain', sep = ':', names = ['a', 'b'])

dataset = pd.read_csv('cal_housing.data', names = list(header['a']))


dataset['roomsPerHousehold'] = dataset['totalRooms']/dataset['households']
dataset['populationPerHousehold'] = dataset['population']/dataset['households']
dataset['bedroomsPerRoom'] = dataset['totalBedrooms']/dataset['totalRooms']
dataset['medianHouseValue'] /= 100000


X = dataset.drop(['medianHouseValue'], axis = 1).values
y = dataset['medianHouseValue'].values
X, y


(array([[-122.23      ,   37.88      ,   41.        , ...,    6.98412698,
            2.55555556,    0.14659091],
        [-122.22      ,   37.86      ,   21.        , ...,    6.23813708,
            2.10984183,    0.15579659],
        [-122.24      ,   37.85      ,   52.        , ...,    8.28813559,
            2.80225989,    0.12951602],
        ...,
        [-121.22      ,   39.43      ,   17.        , ...,    5.20554273,
            2.3256351 ,    0.21517303],
        [-121.32      ,   39.43      ,   18.        , ...,    5.32951289,
            2.12320917,    0.21989247],
        [-121.24      ,   39.37      ,   16.        , ...,    5.25471698,
            2.61698113,    0.22118492]]),
 array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score, mean_squared_error


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

start2 = time.time()
regressor = ElasticNet(alpha=0.8, l1_ratio= 0.3, fit_intercept=True, normalize=False, precompute=False, max_iter=1000, copy_X=True)
#regressor = LinearRegression(fit_intercept=True, normalize=True, copy_X=True, n_jobs=100)
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
m2 = mean_squared_error(y_test, y_pred)

r22 = r2_score(y_test, y_pred)

print(m2, r22)
end2 = time.time()

sp2 = end2 - start2

print(f"Runtime of the program is {end2 - start2} seconds")

0.6474353256685227 0.5034833455365693
Runtime of the program is 0.07802224159240723 seconds


In [ ]:
spark.stop()
s = dict()
s["Framework"] = ["PySpark","Sklearn"]
s["Fit_time"] = [sp, sp2]
s['MSE'] = [m1, m2]
s['Rsquared'] = [r1, r22]

pd.DataFrame(s)

,Framework,Fit_time,MSE,Rsquared
0,PySpark,4.694942,0.773403,0.419246
1,Sklearn,0.078022,0.647435,0.503483
